In [103]:
import psycopg2
import psycopg2.extras as extras
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

## Connect to RDS and Query Data

In [10]:
conn = psycopg2.connect(
    
    host="host",
    database="db",
    user="user",
    password="password" 
)

cur = conn.cursor()
cur.execute('''

            SELECT 
              s.*, r.diagnosis
            FROM  
              bcwis.samples s
            INNER JOIN 
              bcwis.results r
            ON 
              s.id = r.id

            ''')
results = cur.fetchall()
features = [desc[0] for desc in cur.description]
conn.close()

## Convert Query Results to Pandas DataFrame

In [12]:
df = pd.DataFrame(results, columns=features)

### Check Class Balance

In [16]:
df['diagnosis'].value_counts()

B    357
M    212
Name: diagnosis, dtype: int64

### Split into X and y

In [64]:
X = df.iloc[:,1:-1].values

y = df.iloc[:,-1].values

### Stratified Kfold Splits

Helps in handling the imbalanced data for training.

In [70]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

for train_ix, test_ix in kfold.split(X, y):
	# select rows
	train_X, test_X = X[train_ix], X[test_ix]
	train_y, test_y = y[train_ix], y[test_ix]
	# # summarize train and test composition
	train_0, train_1 = len(train_y[train_y=='M']), len(train_y[train_y=='B'])
	test_0, test_1 = len(test_y[test_y=='M']), len(test_y[test_y=='B'])
	print('>Train: M=%d, B=%d, Test: M=%d, B=%d' % (train_0, train_1, test_0, test_1))

>Train: M=169, B=286, Test: M=43, B=71
>Train: M=169, B=286, Test: M=43, B=71
>Train: M=170, B=285, Test: M=42, B=72
>Train: M=170, B=285, Test: M=42, B=72
>Train: M=170, B=286, Test: M=42, B=71


## Fit a Random Forest Classifier

In [71]:
rfc = RandomForestClassifier(max_depth=2, random_state=0)
rfc.fit(train_X, train_y)

RandomForestClassifier(max_depth=2, random_state=0)

## Predict Classes for Test Set

In [80]:
predictions = rfc.predict(test_X)

## Predict All Samples and Probabilities

In [129]:
totalpredictions = rfc.predict(df.iloc[:,1:-1].values)
probabilities = np.max(np.round(rfc.predict_proba(df.iloc[:,1:-1].values),2), axis=1)

In [131]:
resultdf = pd.DataFrame(np.column_stack([df['id'],df['diagnosis'],totalpredictions,probabilities]), columns=['id','diagnosis','prediction','probability'])
resultdf['correct'] = (resultdf['diagnosis'] == resultdf['prediction']) 
resultdf['algorithm'] = 'Random Forest Classifier'
resultdf

,id,diagnosis,prediction,probability,correct,algorithm
0,842302,M,M,0.91,True,Random Forest Classifier
1,842517,M,M,0.92,True,Random Forest Classifier
2,84300903,M,M,0.98,True,Random Forest Classifier
3,84348301,M,M,0.63,True,Random Forest Classifier
4,84358402,M,M,0.93,True,Random Forest Classifier
...,...,...,...,...,...,...
564,926424,M,M,0.98,True,Random Forest Classifier
565,926682,M,M,0.97,True,Random Forest Classifier
566,926954,M,M,0.87,True,Random Forest Classifier
567,927241,M,M,0.98,True,Random Forest Classifier


### Write Prediction Results Back to RDS

In [132]:
tuples = [tuple(x) for x in resultdf.to_numpy()]
query = "INSERT INTO %s(%s) VALUES %%s" % ('bcwis.prediction_results', 'id,ground_truth,prediction,probability,correct,algorithm')

conn = psycopg2.connect(
    
    host="host",
    database="db",
    user="user",
    password="pass" 
)

cursor = conn.cursor()
extras.execute_values(cursor, query, tuples)
conn.commit()
cursor.close()

### Write Algo/Experiment Details to RDS

In [143]:
algorithm = 'Random Forest Classifier'
accuracy = round(metrics.accuracy_score(test_y, predictions),3)
notes = 'Stratified Kfold with 5 folds was used to support the class imbalance. Standard SK Learn random forest classifier with depth of two.'

modeltuple = [tuple(np.array([algorithm,accuracy,notes]))]

query = "INSERT INTO %s(%s) VALUES %%s" % ('bcwis.algo_details', 'algorithm,accuracy,notes')

conn = psycopg2.connect(
    
    host="host",
    database="db",
    user="user",
    password="pass" 
)

cursor = conn.cursor()
extras.execute_values(cursor, query, modeltuple)
conn.commit()
cursor.close()